## Prototype of CNN model used in this paper: https://www.sciencedirect.com/science/article/pii/S2352914818301333

In [1]:
## Set environment variable for data reading
import os
os.environ["PTB_FOLDER_PATH"] = "/Users/cesargenatios/Downloads/ptb-diagnostic-ecg-database-1.0.0"

In [2]:
## Load data from PTB dataset
import sys
sys.path.insert(0,"/Users/cesargenatios/Desktop/cappy/ptd_data_handler")
from data_formatter import *
all_patient_data = get_formatted_ptb_data()


In [3]:
## Choose two patients one with MI, one with Non-MI to test PoC. Test on another 
mi_train_df = all_patient_data['patient024']['s0094lre']
healthy_train_df = all_patient_data['patient252']['s0487_re']

mi_test_df = all_patient_data['patient024']['s0083lre']

In [4]:
del all_patient_data

In [27]:
import numpy as np
## Grab 512 of one patient data
mi_train_data = mi_train_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']].to_numpy()
mi_train_data = np.reshape(mi_train_data, [45,5,512,12])
mi_train_label = np.ones((45,1,1))
print(mi_train_data.shape, mi_train_label.shape) 

mi_test_data = mi_test_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']].to_numpy()
mi_test_data = np.reshape(mi_test_data, [45,5,512,12])
mi_test_label = np.ones((45,1,1))
print(mi_test_data.shape, mi_test_label.shape)

(45, 5, 512, 12) (45, 1, 1)
(45, 5, 512, 12) (45, 1, 1)


## Model Architecture

In [22]:
import tensorflow as tf

seq_length = 5
beat_length = 512
num_feats = 12

## Layer 0 - input
input = tf.keras.Input(shape=(seq_length, beat_length, num_feats))

## Layers 1-6 - time-distributed convolutional block
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(input)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool1D(pool_size=2))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5))(x)

## Layers 7-12 - time-distributed convolutional block
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool1D(pool_size=2))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5))(x)

## Layers 13-19 - time-distributed convolutional block
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool1D(pool_size=2))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5))(x)

## Layers 19-24 - time-distributed convolutional block
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool1D(pool_size=2))(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5))(x)

## Layer 25 - time-distributed flatten
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(x)

## Layer 26 - time-distributed dense
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(32, activation='relu'))(x)

## Layer 27 - time-distributed batch norm
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)

## Layer 28 - time-distributed dropout
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5))(x)

## Layer 29 - LSTM
x = tf.keras.layers.LSTM(32)(x)

## Layer 30 - batch norm
x = tf.keras.layers.BatchNormalization()(x)

## Layer 31 - dropout
x = tf.keras.layers.Dropout(0.5)(x)

## Layer 32 - dense
x = tf.keras.layers.Dense(16, activation='relu')(x)

## Layer 33 - batch norm
x = tf.keras.layers.BatchNormalization()(x)

## Layer 34 - dropout
x = tf.keras.layers.Dropout(0.5)(x)

## Layer 35 - output
output = tf.keras.layers.Dense(1, activation='softmax')(x)

cnn_lstm_model = tf.keras.Model(input, output, name="cnn_lstm_model")
cnn_lstm_model.summary()

Model: "cnn_lstm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 5, 512, 12)]      0         
_________________________________________________________________
time_distributed_58 (TimeDis (None, 5, 512, 32)        1184      
_________________________________________________________________
time_distributed_59 (TimeDis (None, 5, 512, 32)        128       
_________________________________________________________________
time_distributed_60 (TimeDis (None, 5, 512, 32)        3104      
_________________________________________________________________
time_distributed_61 (TimeDis (None, 5, 512, 32)        128       
_________________________________________________________________
time_distributed_62 (TimeDis (None, 5, 256, 32)        0         
_________________________________________________________________
time_distributed_63 (TimeDis (None, 5, 256, 32)     

In [23]:
tf.keras.utils.plot_model(cnn_lstm_model, "cnn_lstm_model.png", show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [24]:
cnn_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(), metrics=tf.keras.metrics.BinaryAccuracy())

In [28]:
tf.config.experimental_run_functions_eagerly(True)

cnn_lstm_model.fit(x=mi_train_data, y=mi_train_label, batch_size=32, epochs=2, verbose=True, validation_data=(mi_test_data, mi_test_label))

Epoch 1/2
2/2 [==============================] - 2s 1s/step - loss: 0.9808 - binary_accuracy: 1.0000 - val_loss: 0.6677 - val_binary_accuracy: 1.0000
Epoch 2/2
2/2 [==============================] - 2s 933ms/step - loss: 1.0874 - binary_accuracy: 1.0000 - val_loss: 0.6523 - val_binary_accuracy: 1.0000


In [29]:
tf.saved_model.save(cnn_lstm_model, "/Users/cesargenatios/Desktop/cappy/model_dev/model_prototypes/cnn_lstm/saved_model")

INFO:tensorflow:Assets written to: /Users/cesargenatios/Desktop/cappy/model_dev/model_prototypes/cnn_lstm/saved_model/assets


INFO:tensorflow:Assets written to: /Users/cesargenatios/Desktop/cappy/model_dev/model_prototypes/cnn_lstm/saved_model/assets
